## 1.1 GPU & VRAM Detection

First, we verify the GPU and available VRAM to determine optimal training settings.

In [ ]:
import subprocess
import torch

def get_gpu_info():
    """Detect GPU and available VRAM"""
    if torch.cuda.is_available():
        gpu_name = torch.cuda.get_device_name(0)
        vram_gb = torch.cuda.get_device_properties(0).total_memory / 1e9
        print(f"✓ GPU Detected: {gpu_name}")
        print(f"✓ Total VRAM: {vram_gb:.2f} GB")

        # Determine recommended settings
        if vram_gb < 16:
            print("⚠ Low VRAM detected. Will use aggressive optimizations.")
            return "low"
        elif vram_gb < 24:
            print("✓ Medium VRAM. Standard LoRA training recommended.")
            return "medium"
        else:
            print("✓ High VRAM. Full fine-tuning possible.")
            return "high"
    else:
        print("✗ No GPU detected. This notebook requires a GPU.")
        return None

vram_tier = get_gpu_info()

## 1.2 Dependency Installation

Installing exact versions for reproducibility and compatibility.

In [ ]:
# Install core dependencies
!pip install -q torch==2.1.0 torchvision==0.16.0 --index-url https://download.pytorch.org/whl/cu118
!pip install -q diffusers[torch]==0.30.3
!pip install -q transformers==4.44.2
!pip install -q accelerate==0.34.2
!pip install -q xformers==0.0.22.post7
!pip install -q bitsandbytes==0.43.3
!pip install -q peft==0.12.0
!pip install -q datasets==2.20.0
!pip install -q Pillow==10.4.0
!pip install -q safetensors==0.4.4

print("✓ All dependencies installed successfully")

## 1.3 Configuration Setup

Set up Hugging Face authentication and global configuration.

In [ ]:
from huggingface_hub import login
import os

# === CONFIGURATION ===
HF_TOKEN = "YOUR_HUGGINGFACE_TOKEN_HERE"  # Replace with your token
MODEL_ID = "stabilityai/stable-diffusion-3.5-large"  # or stable-diffusion-3.5-medium
OUTPUT_DIR = "/content/drive/MyDrive/sd35_training"  # Google Drive path
DATASET_NAME = "lambdalabs/naruto-blip-captions"  # Example dataset

# Authenticate with Hugging Face
if HF_TOKEN != "YOUR_HUGGINGFACE_TOKEN_HERE":
    login(token=HF_TOKEN)
    print("✓ Authenticated with Hugging Face")
else:
    print("⚠ Please set your HF_TOKEN to download gated models")

# Mount Google Drive for checkpoint saving
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Create output directory
os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"✓ Output directory: {OUTPUT_DIR}")

## 2. Stable Diffusion 3.5 Architecture Overview

### Key Components:

**1. Multimodal Diffusion Transformer (MMDiT)**
- Replaces traditional UNet with a transformer-based architecture
- Uses separate pathways for text and image features
- Bidirectional information flow between modalities
- More parameter-efficient than SDXL

**2. Three Text Encoders**
- CLIP ViT-L/14: Visual-semantic alignment
- CLIP ViT-G/14: Enhanced semantic understanding
- T5-XXL: Superior text comprehension and prompt adherence

**3. Rectified Flow Framework**
- Replaces traditional diffusion process
- Straight-line trajectories in latent space (vs. curved diffusion paths)
- Fewer sampling steps required (4-8 vs. 20-50)
- More stable training dynamics

**4. Variational Autoencoder (VAE)**
- 16-channel latent space (vs. 4 in SDXL)
- Higher fidelity reconstruction
- Better detail preservation

### SD3.5 vs SDXL:

| Feature | SDXL | SD3.5 |
|---------|------|-------|
| Architecture | UNet | Transformer (MMDiT) |
| Text Encoders | 2 (CLIP-L, CLIP-G) | 3 (+T5-XXL) |
| Sampling Steps | 20-50 | 4-8 |
| Latent Channels | 4 | 16 |
| Prompt Adherence | Good | Excellent |
| Fine Detail | Good | Superior |

### Why SD3.5 for Fine-tuning:

1. **Better Prompt Understanding**: T5-XXL encoder enables complex prompt following
2. **Faster Convergence**: Rectified flow training is more stable
3. **Modular Fine-tuning**: Can target specific components (text encoders, MMDiT)
4. **LoRA-Friendly**: Transformer architecture benefits greatly from LoRA

## 3.1 Dataset Loading Strategies

Three options for loading training data in Colab:
1. **Hugging Face Datasets** (recommended for public datasets)
2. **Google Drive** (for private datasets)
3. **Direct Upload** (for small datasets)

In [ ]:
from datasets import load_dataset
from torch.utils.data import Dataset
from PIL import Image
import io

class SD35Dataset(Dataset):
    """Custom dataset for SD3.5 training with flexible loading"""

    def __init__(self, dataset_source, resolution=1024, center_crop=True):
        """
        Args:
            dataset_source: HF dataset name, Drive path, or list of image paths
            resolution: Target image resolution (will bucket nearby sizes)
            center_crop: Whether to center crop images
        """
        self.resolution = resolution
        self.center_crop = center_crop

        # Load dataset based on source type
        if isinstance(dataset_source, str):
            if "/" in dataset_source:  # Hugging Face dataset
                print(f"Loading HF dataset: {dataset_source}")
                self.hf_dataset = load_dataset(dataset_source, split="train")
                self.data_type = "hf"
            else:  # Google Drive path
                print(f"Loading from Drive: {dataset_source}")
                self.load_from_drive(dataset_source)
                self.data_type = "drive"
        else:  # List of paths
            self.image_paths = dataset_source
            self.data_type = "paths"

    def load_from_drive(self, drive_path):
        """Load images from Google Drive folder"""
        import glob
        self.image_paths = glob.glob(f"{drive_path}/**/*.jpg", recursive=True)
        self.image_paths += glob.glob(f"{drive_path}/**/*.png", recursive=True)
        print(f"Found {len(self.image_paths)} images")

    def __len__(self):
        if self.data_type == "hf":
            return len(self.hf_dataset)
        else:
            return len(self.image_paths)

    def __getitem__(self, idx):
        # Load image based on source type
        if self.data_type == "hf":
            item = self.hf_dataset[idx]
            image = item["image"]
            caption = item.get("text", item.get("caption", ""))
        else:
            image = Image.open(self.image_paths[idx]).convert("RGB")
            # Try to load caption from .txt file with same name
            txt_path = self.image_paths[idx].rsplit(".", 1)[0] + ".txt"
            try:
                with open(txt_path, "r") as f:
                    caption = f.read().strip()
            except:
                caption = ""

        # Preprocess image
        image = self.preprocess_image(image)

        return {
            "image": image,
            "caption": caption
        }

    def preprocess_image(self, image):
        """Preprocess image with resolution bucketing"""
        # Calculate aspect ratio bucket
        w, h = image.size
        aspect_ratio = w / h

        # Define buckets (common aspect ratios)
        buckets = [
            (1024, 1024),  # 1:1
            (1152, 896),   # 9:7
            (896, 1152),   # 7:9
            (1216, 832),   # 3:2
            (832, 1216),   # 2:3
        ]

        # Find closest bucket
        target_w, target_h = min(buckets,
            key=lambda b: abs(b[0]/b[1] - aspect_ratio))

        # Resize and crop
        if self.center_crop:
            # Resize to slightly larger than target
            scale = max(target_w / w, target_h / h)
            new_w, new_h = int(w * scale), int(h * scale)
            image = image.resize((new_w, new_h), Image.LANCZOS)

            # Center crop to target
            left = (new_w - target_w) // 2
            top = (new_h - target_h) // 2
            image = image.crop((left, top, left + target_w, top + target_h))
        else:
            image = image.resize((target_w, target_h), Image.LANCZOS)

        return image

# Load dataset
print("Loading dataset...")
train_dataset = SD35Dataset(DATASET_NAME, resolution=1024)
print(f"✓ Dataset loaded: {len(train_dataset)} samples")

## 3.2 Dataset Visualization

Verify that images and captions are loading correctly.

In [ ]:
import matplotlib.pyplot as plt

def visualize_batch(dataset, num_samples=4):
    """Display sample images with captions"""
    fig, axes = plt.subplots(1, num_samples, figsize=(20, 5))

    for i in range(num_samples):
        sample = dataset[i]
        axes[i].imshow(sample["image"])
        axes[i].set_title(sample["caption"][:50] + "...", fontsize=10)
        axes[i].axis("off")

    plt.tight_layout()
    plt.show()

visualize_batch(train_dataset)

## 4. Training Strategy Selection

### Option 1: LoRA (Low-Rank Adaptation) ⭐ RECOMMENDED
**VRAM**: 10-16 GB
**Speed**: Fast
**Quality**: Excellent for style/subject fine-tuning
**Use Case**: Most scenarios

- Trains low-rank decomposition matrices (typically rank 4-128)
- Only 0.5-5% of original parameters
- Minimal quality loss for targeted adaptations
- Easy to merge/share

### Option 2: Partial Fine-tuning
**VRAM**: 20-40 GB
**Speed**: Medium
**Quality**: Better for drastic style changes
**Use Case**: Significant distribution shift

- Fine-tune only transformer blocks or text encoders
- More expressive than LoRA
- Requires more VRAM and time

### Option 3: Full Fine-tuning
**VRAM**: 40+ GB
**Speed**: Slow
**Quality**: Maximum control
**Use Case**: Large datasets, new domains

- Not recommended for Colab due to VRAM limits

---

**We'll implement LoRA training with automatic VRAM optimization.**

In [ ]:
from dataclasses import dataclass

@dataclass
class TrainingConfig:
    """Training hyperparameters optimized for Colab"""

    # Model
    model_id: str = MODEL_ID

    # LoRA settings
    lora_rank: int = 16  # 4-128, higher = more expressive
    lora_alpha: int = 16  # Scaling factor, usually = rank
    lora_dropout: float = 0.0
    target_modules: list = None  # Auto-detect transformer blocks

    # Training
    num_epochs: int = 10
    train_batch_size: int = 1  # Per GPU
    gradient_accumulation_steps: int = 4  # Effective batch = 4
    learning_rate: float = 1e-4
    lr_scheduler: str = "cosine"  # or "constant", "linear"
    warmup_steps: int = 100

    # Optimization
    mixed_precision: str = "fp16"  # or "bf16" for A100
    gradient_checkpointing: bool = True
    use_8bit_adam: bool = True  # bitsandbytes optimizer
    max_grad_norm: float = 1.0

    # Data
    resolution: int = 1024
    dataloader_num_workers: int = 2

    # Checkpointing
    save_steps: int = 250
    checkpointing_steps: int = 500
    output_dir: str = OUTPUT_DIR

    # Logging
    logging_steps: int = 10
    report_to: str = "tensorboard"  # or "wandb"

    # Validation
    validation_prompt: str = "a beautiful landscape with mountains"
    num_validation_images: int = 2
    validation_steps: int = 250

# Auto-adjust based on VRAM
config = TrainingConfig()

if vram_tier == "low":
    config.train_batch_size = 1
    config.gradient_accumulation_steps = 8
    config.lora_rank = 8
    config.resolution = 768
    print("⚙ Using low-VRAM settings")
elif vram_tier == "medium":
    config.train_batch_size = 1
    config.gradient_accumulation_steps = 4
    config.lora_rank = 16
    print("⚙ Using medium-VRAM settings")
else:
    config.train_batch_size = 2
    config.gradient_accumulation_steps = 2
    config.lora_rank = 32
    print("⚙ Using high-VRAM settings")

print(f"Effective batch size: {config.train_batch_size * config.gradient_accumulation_steps}")

## 5.1 Model Loading with Memory Optimization

Load SD3.5 with quantization and gradient checkpointing.

In [ ]:
from diffusers import StableDiffusion3Pipeline, FlowMatchEulerDiscreteScheduler
from peft import LoraConfig, get_peft_model
import torch

def load_sd35_for_training(model_id, config):
    """Load SD3.5 with memory optimizations"""

    print("Loading Stable Diffusion 3.5...")

    # Load pipeline with optimizations
    pipe = StableDiffusion3Pipeline.from_pretrained(
        model_id,
        torch_dtype=torch.float16 if config.mixed_precision == "fp16" else torch.bfloat16,
        variant="fp16",
        use_safetensors=True,
    )

    # Enable memory optimizations
    if config.gradient_checkpointing:
        pipe.transformer.enable_gradient_checkpointing()
        print("✓ Gradient checkpointing enabled")

    # Enable xformers for memory-efficient attention
    try:
        pipe.enable_xformers_memory_efficient_attention()
        print("✓ xFormers enabled")
    except:
        print("⚠ xFormers not available")

    # Move to GPU
    pipe = pipe.to("cuda")

    return pipe

# Load model
pipeline = load_sd35_for_training(config.model_id, config)

## 5.2 LoRA Configuration

Configure LoRA adapters for the MMDiT transformer.

In [ ]:
def setup_lora(pipeline, config):
    """Apply LoRA to SD3.5 transformer"""

    # Configure LoRA for transformer blocks
    lora_config = LoraConfig(
        r=config.lora_rank,
        lora_alpha=config.lora_alpha,
        init_lora_weights="gaussian",
        target_modules=[
            "to_q", "to_k", "to_v", "to_out.0",  # Attention layers
            "proj_in", "proj_out",  # Projection layers
        ],
        lora_dropout=config.lora_dropout,
    )

    # Apply LoRA to transformer
    pipeline.transformer = get_peft_model(pipeline.transformer, lora_config)

    # Print trainable parameters
    trainable_params = sum(p.numel() for p in pipeline.transformer.parameters() if p.requires_grad)
    all_params = sum(p.numel() for p in pipeline.transformer.parameters())

    print(f"✓ LoRA configured")
    print(f"  Trainable params: {trainable_params:,} ({100 * trainable_params / all_params:.2f}%%)")
    print(f"  Total params: {all_params:,}")

    return pipeline

# Setup LoRA
pipeline = setup_lora(pipeline, config)

## 5.3 Training Loop Implementation

Full training loop with accelerate integration.

In [ ]:
from accelerate import Accelerator
from accelerate.utils import ProjectConfiguration
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
import torch.nn.functional as F
from diffusers.optimization import get_scheduler
import bitsandbytes as bnb

def collate_fn(examples):
    """Custom collate function for batching"""
    images = [example["image"] for example in examples]
    captions = [example["caption"] for example in examples]
    return {"images": images, "captions": captions}

def train_sd35_lora(pipeline, train_dataset, config):
    """Complete training loop for SD3.5 LoRA"""

    # Initialize accelerator
    accelerator_project_config = ProjectConfiguration(
        project_dir=config.output_dir,
        logging_dir=f"{config.output_dir}/logs"
    )

    accelerator = Accelerator(
        gradient_accumulation_steps=config.gradient_accumulation_steps,
        mixed_precision=config.mixed_precision,
        log_with=config.report_to,
        project_config=accelerator_project_config,
    )

    # Create dataloader
    train_dataloader = DataLoader(
        train_dataset,
        batch_size=config.train_batch_size,
        shuffle=True,
        collate_fn=collate_fn,
        num_workers=config.dataloader_num_workers,
    )

    # Prepare optimizer
    if config.use_8bit_adam:
        optimizer_cls = bnb.optim.AdamW8bit
    else:
        optimizer_cls = torch.optim.AdamW

    optimizer = optimizer_cls(
        pipeline.transformer.parameters(),
        lr=config.learning_rate,
        betas=(0.9, 0.999),
        weight_decay=1e-2,
        eps=1e-8,
    )

    # Prepare scheduler
    lr_scheduler = get_scheduler(
        config.lr_scheduler,
        optimizer=optimizer,
        num_warmup_steps=config.warmup_steps * config.gradient_accumulation_steps,
        num_training_steps=len(train_dataloader) * config.num_epochs,
    )

    # Prepare with accelerator
    pipeline.transformer, optimizer, train_dataloader, lr_scheduler = accelerator.prepare(
        pipeline.transformer, optimizer, train_dataloader, lr_scheduler
    )

    # Training loop
    global_step = 0
    progress_bar = tqdm(
        range(config.num_epochs * len(train_dataloader)),
        disable=not accelerator.is_local_main_process,
    )
    progress_bar.set_description("Training")

    for epoch in range(config.num_epochs):
        pipeline.transformer.train()

        for step, batch in enumerate(train_dataloader):
            with accelerator.accumulate(pipeline.transformer):
                # Encode images to latents
                images = [img for img in batch["images"]]
                captions = batch["captions"]

                # Process images through VAE
                latents = []
                for img in images:
                    # Convert PIL to tensor
                    img_tensor = torch.tensor(
                        np.array(img).transpose(2, 0, 1)
                    ).unsqueeze(0).float() / 255.0
                    img_tensor = img_tensor.to(accelerator.device, dtype=pipeline.vae.dtype)

                    # Encode
                    with torch.no_grad():
                        latent = pipeline.vae.encode(img_tensor).latent_dist.sample()
                        latent = latent * pipeline.vae.config.scaling_factor
                    latents.append(latent)

                latents = torch.cat(latents, dim=0)

                # Sample noise and timesteps
                noise = torch.randn_like(latents)
                bsz = latents.shape[0]

                # Rectified flow uses timesteps in [0, 1]
                timesteps = torch.rand(bsz, device=latents.device)

                # Add noise (linear interpolation for rectified flow)
                noisy_latents = (1 - timesteps.view(-1, 1, 1, 1)) * latents + \
                               timesteps.view(-1, 1, 1, 1) * noise

                # Encode text
                with torch.no_grad():
                    prompt_embeds, pooled_prompt_embeds = pipeline.encode_prompt(
                        prompt=captions,
                        prompt_2=captions,
                        prompt_3=captions,
                        device=accelerator.device,
                    )

                # Predict the noise (velocity for rectified flow)
                model_pred = pipeline.transformer(
                    hidden_states=noisy_latents,
                    timestep=timesteps,
                    encoder_hidden_states=prompt_embeds,
                    pooled_projections=pooled_prompt_embeds,
                    return_dict=False,
                )[0]

                # Rectified flow loss: predict velocity (noise - latents)
                target = noise - latents
                loss = F.mse_loss(model_pred.float(), target.float(), reduction="mean")

                # Backprop
                accelerator.backward(loss)

                if accelerator.sync_gradients:
                    accelerator.clip_grad_norm_(pipeline.transformer.parameters(), config.max_grad_norm)

                optimizer.step()
                lr_scheduler.step()
                optimizer.zero_grad()

            # Logging
            if accelerator.sync_gradients:
                progress_bar.update(1)
                global_step += 1

                if global_step % config.logging_steps == 0:
                    logs = {
                        "loss": loss.detach().item(),
                        "lr": lr_scheduler.get_last_lr()[0],
                        "step": global_step,
                    }
                    progress_bar.set_postfix(**logs)
                    accelerator.log(logs, step=global_step)

                # Save checkpoint
                if global_step % config.checkpointing_steps == 0:
                    if accelerator.is_main_process:
                        save_path = f"{config.output_dir}/checkpoint-{global_step}"
                        accelerator.save_state(save_path)
                        print(f"✓ Checkpoint saved: {save_path}")

                # Generate validation images
                if global_step % config.validation_steps == 0:
                    if accelerator.is_main_process:
                        generate_validation_images(
                            pipeline, config, global_step, accelerator
                        )

    # Save final LoRA weights
    accelerator.wait_for_everyone()
    if accelerator.is_main_process:
        pipeline.transformer.save_pretrained(f"{config.output_dir}/final_lora")
        print(f"✓ Final LoRA saved: {config.output_dir}/final_lora")

    accelerator.end_training()

def generate_validation_images(pipeline, config, step, accelerator):
    """Generate validation images during training"""
    pipeline.transformer.eval()

    with torch.no_grad():
        images = pipeline(
            prompt=config.validation_prompt,
            num_inference_steps=28,
            guidance_scale=7.0,
            num_images_per_prompt=config.num_validation_images,
        ).images

    # Save images
    for i, img in enumerate(images):
        img.save(f"{config.output_dir}/validation-{step}-{i}.png")

    pipeline.transformer.train()

import numpy as np

# Start training
print("🚀 Starting training...")
train_sd35_lora(pipeline, train_dataset, config)

## 6.1 Loading Trained Model

Load the base model and merge LoRA weights for inference.

In [ ]:
def load_trained_model(base_model_id, lora_path, device="cuda"):
    """Load base model and merge LoRA weights"""

    print("Loading model for inference...")

    # Load base pipeline
    pipe = StableDiffusion3Pipeline.from_pretrained(
        base_model_id,
        torch_dtype=torch.float16,
        variant="fp16",
        use_safetensors=True,
    )

    # Load and merge LoRA weights
    pipe.load_lora_weights(lora_path)

    # Enable optimizations
    pipe.enable_xformers_memory_efficient_attention()
    pipe.to(device)

    print("✓ Model loaded successfully")
    return pipe

# Load trained model
inference_pipe = load_trained_model(
    config.model_id,
    f"{config.output_dir}/final_lora"
)

## 6.2 Image Generation

Generate images with various samplers and settings.

In [ ]:
def generate_images(
    pipe,
    prompt,
    negative_prompt="blurry, bad quality, distorted",
    num_images=4,
    guidance_scale=7.0,
    num_inference_steps=28,
    height=1024,
    width=1024,
    seed=None,
):
    """Generate images with SD3.5"""

    if seed is not None:
        generator = torch.Generator(device="cuda").manual_seed(seed)
    else:
        generator = None

    images = pipe(
        prompt=prompt,
        negative_prompt=negative_prompt,
        num_inference_steps=num_inference_steps,
        guidance_scale=guidance_scale,
        height=height,
        width=width,
        num_images_per_prompt=num_images,
        generator=generator,
    ).images

    return images

# Example generation
prompt = "a serene Japanese garden with cherry blossoms, koi pond, and traditional architecture, highly detailed"
images = generate_images(inference_pipe, prompt, num_images=2, seed=42)

# Display
fig, axes = plt.subplots(1, len(images), figsize=(15, 8))
for i, img in enumerate(images):
    if len(images) == 1:
        axes.imshow(img)
        axes.axis("off")
    else:
        axes[i].imshow(img)
        axes[i].axis("off")
plt.suptitle(prompt[:100])
plt.tight_layout()
plt.show()

## 6.3 Advanced Sampling

Compare different schedulers and settings.

In [ ]:
from diffusers import (
    FlowMatchEulerDiscreteScheduler,
    DPMSolverMultistepScheduler,
    DDIMScheduler,
)

def compare_schedulers(pipe, prompt, schedulers_config):
    """Compare different noise schedulers"""

    results = {}
    original_scheduler = pipe.scheduler

    for name, scheduler_cls, steps in schedulers_config:
        print(f"Testing {name}...")
        pipe.scheduler = scheduler_cls.from_config(pipe.scheduler.config)

        images = generate_images(
            pipe, prompt,
            num_images=1,
            num_inference_steps=steps,
            seed=42
        )
        results[name] = images[0]

    pipe.scheduler = original_scheduler
    return results

# Test different schedulers
schedulers = [
    ("Euler (Default)", FlowMatchEulerDiscreteScheduler, 28),
    ("DPM++ (Fast)", DPMSolverMultistepScheduler, 20),
    ("DDIM (Quality)", DDIMScheduler, 50),
]

test_prompt = "a cyberpunk city at night, neon lights, rain, cinematic"
scheduler_results = compare_schedulers(inference_pipe, test_prompt, schedulers)

# Display comparison
fig, axes = plt.subplots(1, len(scheduler_results), figsize=(18, 6))
for i, (name, img) in enumerate(scheduler_results.items()):
    axes[i].imshow(img)
    axes[i].set_title(name)
    axes[i].axis("off")
plt.suptitle(test_prompt[:100])
plt.tight_layout()
plt.show()